In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
from evalset.evalset import test_funcs, icml2016_tests
import sys, inspect

from psoas import swarm
from psoas import optimizer as opti

# Evaluation on evalset

In [ ]:
icml = icml2016_tests.tests_for_nonparametric
for idx, test in enumerate(icml):

    test_function_class = getattr(test_funcs, test['name'])
    test_function_base = test_function_class(test['dim'])
    print("########################################################################################################")
    print("Testfunction:", test['name'])
    print("Testfunc.dim:",test_function_base.dim)
    
    try:   
        opt = optimizer.Optimizer(test_function_base.do_evaluate, max(20, test_function_base.dim), test_function_base.dim, np.array(test_function_base.bounds), 500)
        result = opt.optimize()
        plt.plot(result['gbest_list'])
        plt.ylim(ymin=0, ymax=25)
        plt.show()
        print("Bounds:",test_function_base.bounds)
        print(f"The expected minimum was found at {result['x_opt']} with a functionvalue of {result['func_opt']}")
        print(f"The real minimum is located at {test_function_base.min_loc} with a functionvalue of {test_function_base.fmin}")
    except (IndexError, ValueError):
        print("Our optimazier couldn't handel the objectiv function, trying the next one.")
    
    

In [ ]:
Plateau = test_funcs.Plateau()

In [ ]:
opt = optimizer.Optimizer(Plateau.do_evaluate, 20, Plateau.dim, np.array(Plateau.bounds), 50)

# %lprun -f optimizer.Optimizer.optimize gbest_list = opt.optimize()
# %timeit gbest_list = opt.optimize()

result = opt.optimize()
plt.plot(result['gbest_list'])
print(result['x_opt'], result['func_opt'])

In [ ]:
opt = opti.Optimizer(Ackley.do_evaluate, 20, Ackley.dim, np.array(Ackley.bounds), 50)

# %lprun -f optimizer.Optimizer.optimize gbest_list = opt.optimize()
# %timeit gbest_list = opt.optimize()

result = opt.optimize()
plt.plot(result['gbest_list'])
print(result['x_opt'], result['func_opt'])

In [ ]:
sw = swarm.Swarm(f, 10, 2, constraints)
# gbest_list = []
# for i in tqdm(range(100)):
#     sw.update_swarm()
#     #print(sw.pbest_position)
#     gbest, gbest_position = sw.compute_gbest()
#     print(gbest)
#     gbest_list.append(gbest)
    
gbest, gbest_position = sw.compute_gbest()
(gbest, gbest_position)

# Evaluation on CEC2013 LSG

In [ ]:
from cec2013lsgo import cec2013

bench = cec2013.Benchmark()
num_funcs = bench.get_num_functions()

In [ ]:
num_funcs = bench.get_num_functions()

for i in range(num_funcs):
    i+=1
    info = bench.get_info(i)
    dim = info['dimension']
    constrainints = np.ones((dim, 2))*np.array([info['lower'], info['upper']])
    func = bench.get_function(i)


    print("########################################################################################################")
    print("Testfunction ID:", i)
    print("Testfunc.dim:", dim)
    
    opt = opti.Optimizer(func, max(40, dim), dim, constrainints, 500)
    result = opt.optimize()
    plt.plot(result['gbest_list'])
    plt.ylim(ymin=0, ymax=25)
    plt.show()
    print(f"Bounds:{dim}*({info['lower'], info['upper']})")
    print(f"Resulting functionvalue: {result['func_opt']}")
    print(f"The real functionvalue:{info['best']}")
    break

# Evaluation CEC 2013 single

In [ ]:
from cec2013single.cec2013single import cec2013 as cec
# CEC13DATA = cec2013single/cec2013single/cec2013_data

In [ ]:
bench = cec.Benchmark()
dim = 2

In [ ]:
for i in range(1,28):
    info = bench.get_info(i,dim)
    constraints = np.ones((dim, 2))*np.array([info['lower'], info['upper']])
    func = bench.get_function(i)


    print(80 * "=")
    print("Testfunction ID:", i)
    print("Testfunc.dim:", dim)
    
    opt = opti.Optimizer(func, max(40, dim*5), dim, constraints, 200)
    result = opt.optimize()
    
    # plt.plot(result['gbest_list'])
    # plt.xlim(xmin =0)
    # plt.title(f"Function {i}")
    # plt.xlabel("Iterations")
    # plt.ylabel("Cost")
    # plt.grid()

    # # plt.ylim(ymin=0, ymax=25)
    # plt.show()
    print(f"Bounds:{dim}*({info['lower'], info['upper']})")
    print(f"Resulting functionvalue: {result['func_opt']}. Converged after {result['iter']} iterations.")
    print(f"The real functionvalue:{info['best']}")
    print(80 * "=")


# Evaluation pyswarms on CEC 2013 single

In [ ]:
def wrapper(x, f):
    n_particles = x.shape[0]
    sol = [f(x[i]) for i in range(n_particles)]
    return np.array(sol) 

In [ ]:
import pyswarms as ps
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
dim = 2
max_bound = 100 * np.ones(dim)
min_bound = - max_bound
bounds = (min_bound, max_bound)

for i in range(1,28):
    info = bench.get_info(i,dim)
    func = bench.get_function(i)


    print(80 * "=")
    print("Testfunction ID:", i)
    print("Testfunc.dim:", dim)
    
    optimizer = ps.single.GlobalBestPSO(n_particles=40, dimensions=2, options=options, bounds=bounds)

    # Perform optimization
    cost, pos = optimizer.optimize(wrapper, iters=200, f=func)

    # plt.plot(optimizer.cost_history)
    # plt.xlim(xmin =0)
    # plt.title(f"Function {i}")
    # plt.xlabel("Iterations")
    # plt.ylabel("Cost")
    # plt.grid()

    # # plt.ylim(ymin=0, ymax=25)
    # plt.show()
    # print(f"Bounds:{dim}*({info['lower'], info['upper']})")
    print(f"Resulting functionvalue: {cost}.")
    print(f"The real functionvalue:{info['best']}")
    print(80 * "=")
